# Preparation

튜토리얼에 시작하기 앞서서 앞전 튜토리얼때 사용했던 헬퍼 함수와 데이터를 좀 불러오겠습니다. 아래로 스크롤 내리시면 되겠습니다.

In [60]:
import numpy as np

from tensorflow.keras import optimizers, losses, metrics, layers, models
from tensorflow.keras.datasets.mnist import load_data

In [61]:
trainset, testset = load_data()

In [62]:
x_train, y_train = trainset
x_test, y_test = testset

In [63]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [64]:
x_train = x_train.reshape(*x_train.shape, 1)
x_test = x_test.reshape(*x_test.shape, 1)

In [65]:
# mean = 0, std = 0.5
x_train = (x_train.astype(np.float32) - 128) / 256
x_test = (x_test.astype(np.float32) - 128) / 256

In [66]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

In [35]:
def shuffle(x, y):
    """
    x, y를 셔플한다.
    
    Arguments:
    ----------
    x : features 데이터 행렬 (N, ...)
    y : 라벨 벡터            (N,)
    
    Returns:
    --------
    x[r] : x를 셔플한 np.array (N, ...)
    y[r] : y를 셔플한 np.array (N,)
    """
    
    n = x.shape[0]
    
    r = np.arange(n)
    np.random.shuffle(r)
    
    return x[r], y[r]

In [36]:
def next_batch(x, y, batch_size):
    """
    x, y를 해당 batch_size만큼 잘라서 배치를 생성해주는 generator
    
    Arguments:
    ----------
    x : features 데이터 행렬 (N, ...)
    y : 라벨 벡터            (N,)
    
    Returns:
    --------
    x_batch : x를 배치단위로 자른 것
    y_batch : y를 배치단위로 자른 것.
    """
    
    n = x.shape[0]
    n_batches = int(np.ceil(n / batch_size))
    
    for b in range(n_batches):
        start = b*batch_size
        end = min(n, (b+1)*batch_size)
        
        yield x[start:end], y[start:end]

In [90]:
@tf.function
def inference(model, criterion, x, y, training=False):
    """
    Forward propagation 함수.
    
    Arguments:
    ----------
    model : tf.keras 모델 객체
    criterion : loss 함수
    x : 데이터 x
    y : 라벨 y
    training : True이면 loss, predicitons, gradient를 계산해서 리턴해주고, False이면 loss와 prediction만 계산해서 리턴해준다.
    
    Returns:
    --------
    loss : 이 배치에 대한 loss
    preds : 이 배치에 대한 prediction 값
    [grads] : trainig=True일때 반환되며, 그래디언트를 계산한 것
    """
    
    if training is True:
        with tf.GradientTape() as tape:
            preds = model(x, training=training)
            loss = criterion(y, preds)
            
        grads = tape.gradient(loss, model.trainable_variables)
        return loss, preds, grads
    
    else:
        preds = model(x,  training=training)
        loss = criterion(y, preds)
        return loss, preds

In [91]:
@tf.function
def backward(optimizer, grads, variables):
    """
    Backward propagation 함수
    
    Arguments:
    ----------
    optimizer : optimizer 객체
    grads : gradients list
    variables : weights들의 list
    """
    
    optimizer.apply_gradients(zip(grads, variables))

# Custum Models & Layers 

앞 전 튜토리얼에서 ```tf.keras.models.Model```을 상속받아서 모델을 생성하는 법으로 CNN을 구현해보았는데요, 이번 튜토리얼에서는 그 방식에 대해 좀 더 자세히 알아보고자 합니다.

(tf는 생략할게요) Keras에서, 모든 신경망 모델은 사실상 ```keras.models.Model```이라고 보시면 될 것 같습니다. ```keras.models.Sequential```또한 사실 ```keras.models.Model```을 상속받고 있습니다.

이것은 keras의 functional API도 똑같습니다. 예를 들어, 다음 코드는 레이어를 먼저 구축하고, 연결된 레이어를 ```models.Model```을 생성하면서 인자로 넘겨주고 있습니다. 이미 레이어가 서로 연결되어 있기 때문에 input 레이어와 output 레이어만 넘겨주게 되면, ```models.Model```객체는 그 사이 레이어를 알 수 있죠. 이 방법에서도 볼 수 있듯이, 결국 keras에서 신경망은 ```models.Model```객체입니다.
```python
inputs = layers.Input(shape=(784,))
x = layers.Dense(128, activation=tf.nn.leaky_relu)(inputs)
x = layers.Dense(128, activation=tf.nn.leaky_relu)(x)
outputs = layers.Dense(10, activation=tf.nn.softmax)(x)

model = models.Model(inputs=inputs, outputs=outputs)
```

Sequential 방법은 더 명확하죠.
```python
model = models.Sequential([
    layers.Dense(128, activation=tf.nn.leaky_relu, input_shape=(784,))
    layers.Dense(128, activation=tf.nn.leaky_relu)
    layers.Dense(10, activation=tf.nn.softmax)
])
```
Sequential 객체를 생성하는 방법이므로, Model객체를 생성하는 것과 같습니다. Sequential은 Model의 하위 클래스이기 때문이죠.

앞서 저의 예제는 Model 클래스를 직접 상속받는 방법이므로 Model객체를 생성하는 것이 됩니다.

따라서, keras에서 신경망을 만든다는 것은 ```keras.models.Model```객체를 만들고, 이 객체에 layer를 구축해 주고, 이 객체를 build, 및 compile해 주는 것입니다. 저는 위에 제시한 세 가지 방법중, 제일 마지막 방법인 Model을 상속받은 하위 클래스를 직접 정의하는 방법을 선호한다고 말씀드리는데요, 그냥 저의 취향입니다. 이것이 더 좋다기보단, 이렇게 코드를 구성하는게 제 기준에서 더 깔끔하고 가독성이 좋아 보입니다. 어떻게 보면, 저는 tensorflow 1.x와 pytorch를 접하고 keras를 접했기 때문에, pytorch에서 하던 방식이 편했을지도 모릅니다. (pytorch도 keras에서 Model에 해당하는 클래스를 상속받아 모델 클래스를 정의하는 방법을 많이 씁니다.)

하지만, 딥러닝에 대한 학습을 진행할수록, 기존의 convolution layer와 recurrent layer, dense layer등의 구조를 그대로 가져다가 쓰기보단, 새로운 구조 및 유닛을 생성하고 테스트해봐야 할 때가 많을것인데요, 이때는 상속받아서 하는게 편하다고 생각합니다. 이번 튜토리얼에서는 ```keras.models.Model```을 상속받아서 custom model을 생성하는 방법을 자세히 알아보고 ```keras.layers.Layer```를 상속받아서 자신만의 레이어를 만드는 방법을 알아보겠습니다!

## Custom Models

```tf.keras.models.Model```을 상속받아서 새로운 모델 클래스를 정의하는 방식은 보통 다음 규격(? 템플릿)을 따릅니다.
```python
from tensorflow.keras import models, layers, losses, metrics, optimizers

class MyModelClass(models.Model):
    
    def __init__(self):
        super(MyModelClass, self).__init__()
        
        # building layers
        
    @tf.function
    def call(self, inputs, training=False):
        
        # forward propagation
        
        return outputs
```
먼저, class 이름은 MyModelClass이죠. 따라서, 나중에 객체를 만드실때, ```mymodel = MyModelClass()```로 만들어 주시면 됩니다.
이 클래스는 ```models.Model```을 상속받게 됩니다.

다음으로, ```def__init__(self)```부분인데, 이것은 파이썬 클래스에서 생성자 역할을 하죠. 우리 생성자는 인자로 ```self```를 받고 있는데, 파이썬 클래스 멤버 메소드는 반드시 첫 번째 인자로 ```self```를 받아야 합니다. ```self```이 클래스로 생성된 객체의 이 메소드를 호출했을 때, 자기 자신(객체)을 의미하는데, 예를 들어, ```MyClass```라는 클래스에 ```def my_method(self)```가 정의되어 있다고 해봅시다. 그리고 ```foo = MyClass()```로 객체를 생성하고 이 객체의 메소드 ```foo.my_method()```를 호출하게 되면, ```my_method```의 첫 인자인 ```self```에는 ```foo```객체 자신이 들어갑니다. 이는 생성자의 경우도 마찬가지 인데요, 파이썬 인터프리터가 객체를 생성하는 코드를 만나게 되면, 객체를 먼저 생성하고 생성자를 호출하게 되는데, 생성한 객체를 생성자의 첫 인자로 넘기게 되죠. 자세한 내용은 파이썬 기본 책자를 찾아보시는게 좋을 것 같습니다. 이 부분은 텐서플로와 거리가 멀죠.

```models.Model```을 상속한 모든 하위 클래스는 반드시 생성자의 첫 task로 ```super(MyModelClass, self).__init__()```을 호출해야 합니다. 이것은 상위 클래스의 생성자를 호출하는 명령으로 Java 클래스 생성자에서 ```super()```와 같은 역할을 합니다. C#의 경우 ```base()```(C#은 안해봐서.. 맞나요?)가 되겠네요. MyModelClass의 상위 클래스는 케라스 Model이므로, 케라스 Model 클래스의 생성자를 호출하는 것인데, 그래야, 생성자 다음 라인에서 정의할 레이어와 변수(weights)들이 케라스 Model에서 인식할 수 있고 trainable로 인식할 수 있게 됩니다. 상위 생성자를 호출하고 나면, 이제 인스턴스 변수(```self.some_vars = ```형식으로 선언한 변수)로 레이어나 ```tf.Variable```을 생성해 주면 됩니다. 이건 잠시후 디테일에서 설명드리죠(지금은 템플릿 설명이니까!).

Model의 가장 기본이 되는 메소드라고 하면 ```def call(self, inputs, tranining=False)```라고 할 수 있는데요. 이 메소드의 역할은 forward propagation입니다. 들어온 inputs 텐서를 우리가 생성자에서 정의한 레이어에 통과시키고 결과를 반환해 주면 됩니다. 또한, forward propagation과정이 training phase와 test phase에서 다를 수 있으니, training 변수를 추가로 받을 수도 있죠.

```call```메소드는 케라스 Model클래스의 ```__call__```메소드에 의해 자동으로 호출됩니다. 즉, 다음처럼 하면 ```call```이 호출됩니다.
```python
inputs = #...

mymodel = MyModelClass()

mymodel(inputs) # mymodel.__call__(inputs) -> mymodel.call(inputs) 순으로 호출됨
```

```__call__```메소드는 파이썬 언어 자체적으로 지원하는 클래스 built-in method인데요, 마치 객체를 함수처럼 쓸 수 있게 해줍니다. 즉, ```객체이름(인자)``` 이렇게 호출이 가능한 것입니다. 케라스 Model은 이 ```__call__```메소드를 override해서 자신의 ```call```메소드를 호출하게끔 해 놓았습니다.(그래서 ```call```메소드 정의안해주면 에러납니다.)

케라스 Model에서 ```call```의 또 다른 역할은 최초로 모델을 빌드할때 이용한다는 것입니다. 케라스 Model의 ```.build(input_shape)```를 호출하게 되면, 내부에서 그 input_shape을 이용하여 테스트 입력을 만드는지 모르겠지만, 어쨌든 ```call```을 호출하더군요. 케라스에서는 입력을 먼저 레이어로 통과시킨 후, 각 레이어의 input_shape를 확정해서 모델을 빌드하는 과정을 거칩니다. 그도 그럴 것이, ```keras.layers```안에 정의된 레이어 클래스를 보시면 레이어의 입력이 어떠한 모양인지 알 수 있는 정보가 ```input_shape```이외엔 없습니다. 그런데, 케라스 코드를 보시면 모든 레이어에 ```input_shape```을 인자로 주지는 않죠. 보통 가장 첫 레이어에만 해당 인자를 줍니다. 즉, 나머지 레이어는 실제로 입력을 직접 주어야 레이어에서 입력의 모양을 확정할 수 있습니다. 방법은 ```call```을 호출하는 것이죠.

케라스 Model을 생성하고 ```call```을 호출하지 않은 상태에서 ```summary```를 호출해 보시면 에러가 납니다. 빌드가 안됬으니 빌드하라고 하는데, 두 가지 방법을 제시합니다. ```build```를 호출하던지, ```call```을 호출하던지 라고 말합니다. 그리고 ```build```를 호출하지 않아도 ```fit```메소드는 잘 작동하고 ```fit```이후에는 ```summary```가 됩니다. ```call```이 호출되면서 모델을 빌드하는 역할도 했기 때문이죠. 하지만, 최초 호출에서만 빌드하는 역할을 하고 그 다음부터는 첫 입력 모양과 같은 inputs 텐서(넘파이 배열도 가능)로 ```call```을 호출하셔야 합니다.

지금까지 전체적인 템플릿을 봤고, 이제 진짜 모델 클래스를 가저와 보도록 하겠습니다. 다음은 이전 튜토리얼에서 사용했던 CNN 모델입니다.

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras import models, layers, losses, metrics, optimizers

class MyCNNModel(models.Model):
    """
    My CNN Model
    """
    
    def __init__(self):
        super(MyCNNModel, self).__init__() # 반드시 생성자 맨 처음에 호출해 줘야 함.
        
        # convolution - pool - convolution - pool
        self.features = models.Sequential([
            layers.Conv2D(8, (3, 3), strides=1, padding="same", input_shape=(28, 28, 1)),
            layers.BatchNormalization(),
            layers.Activation(tf.nn.tanh),
            
            layers.MaxPool2D((2, 2), strides=2, padding="same"),
            
            layers.Conv2D(12, (3, 3), strides=1, padding="same"),
            layers.BatchNormalization(),
            layers.Activation(tf.nn.tanh),
            
            layers.MaxPool2D((2, 2), strides=2, padding="same"),
        ])
        
        # dense - dense - dense
        self.classifier = models.Sequential([
            layers.Dense(64, input_shape=(7*7*12,)),
            layers.Activation(tf.nn.tanh),
            layers.Dropout(0.5),
            
            layers.Dense(32),
            layers.Activation(tf.nn.tanh),
            layers.Dropout(0.5),
            
            layers.Dense(10),
            layers.Activation(tf.nn.softmax)
        ])
        
    @tf.function
    def call(self, inputs, training=False):
        '''
        forward propagation function
        
        Arguments:
        ----------
        inputs : input tensor. (batch_size, 28, 28, 1)
        training: 트레이닝 과정이면 True 넣어줘야 함
        
        Returns:
        --------
        preds : 추론 결과
        '''
        
        n = tf.shape(inputs)[0]
        
        features = self.features(inputs, training=training)
        features = tf.reshape(features, (n, -1))
        
        preds = self.classifier(features, training=training)
        
        return preds

방금 봤던 템플릿에서 달라진 점은 생성자에 ```self.features```와 ```self.classifier```를 생성하는 부분이 생겼고, ```call```내부에 inputs를 실제로 레이어에 통과시키는 과정을 그려놓았다는 것입니다.

먼저, ```self.features```와 ```self.classifier```는 또 다른 케라스 Model입니다(Seqential은 Model입니다!). 이처럼 케라스는 멤버 레이어로 또 다른 모델을 포함할 수 있습니다. 그리고 멤버 모델의 모든 weights 및 bias들도 자신의 weights 및 bias로 삼습니다. 즉, trainable_variables에 들어가게 되죠. 만약, 어떤 멤버 모델의 변수들이 trainiable_variables에 포함되게 하고 싶지 않다면, 해당 멤버 모델의 trainable 변수에 False로 대입해 주면 됩니다.```self.features.trainable = False``` 이렇게 말이죠.

다음을 보죠. 먼저 모델을 생성하고 빌드해 봅시다.

In [6]:
mymodel = MyCNNModel()
mymodel.build(input_shape=(None, 28, 28, 1))

summary를 찍어봅시다.

In [7]:
mymodel.summary()

Model: "my_cnn_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_8 (Sequential)    (None, 7, 7, 12)          1036      
_________________________________________________________________
sequential_9 (Sequential)    (None, 10)                40106     
Total params: 41,142
Trainable params: 41,102
Non-trainable params: 40
_________________________________________________________________


Total params와 Trainable params에 주목해 주세요. 이제, mymodel 객체의 features 서브 모델의 traninable을 False로 만들어 보겠습니다.

In [8]:
mymodel.features.trainable = False
mymodel.summary()

Model: "my_cnn_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_8 (Sequential)    (None, 7, 7, 12)          1036      
_________________________________________________________________
sequential_9 (Sequential)    (None, 10)                40106     
Total params: 41,142
Trainable params: 40,106
Non-trainable params: 1,036
_________________________________________________________________


Trainable params 개수가 조금 줄은게 보이시나요? CNN의 파라미터의 개수는 상당히 작으므로 얼마 안 줄어들었네요. 그래도 features 모델의 모든 파라미터가 Training 대상에서 제외된 것입니다.

그리고, 같은 원리로 모델 전체의 trainable을 끌 수 있습니다.

In [10]:
mymodel.trainable = False
mymodel.summary()

Model: "my_cnn_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_8 (Sequential)    (None, 7, 7, 12)          1036      
_________________________________________________________________
sequential_9 (Sequential)    (None, 10)                40106     
Total params: 41,142
Trainable params: 0
Non-trainable params: 41,142
_________________________________________________________________


Trainable params가 0이 됨을 보실 수 있습니다. 이러면, 모든 weight들이 학습 대상에서 제외됩니다.

이것은 ```call```메소드 내에서 활용될 수 있는데요, training 변수를 각 멤버 모델을 호출할 때 training 변수로 넘겨주면 됩니다. training step에서는 이게 상관없지만, 중요한건 test step일때 겠죠. 딥러닝을 공부해 보셨다면 아시겠지만, Dropout이나 Batch normalization은 training과 test시에 연산이 다릅니다. 따라서 이 과정이 필요하죠. 보시면 다음과 같이 training변수를 넘겨주는 코드를 보실 수 있습니다.
```python
def call(self, inputs, training=False):
    #...
    
    x = self.features(x, training=training)
    
    #...
    
    x = self.classifier(x, training=training)
    
    #...
```

## Custom Layers

커스컴 모델을 정의할 수 있습니다. 그런데, 좀 더 세밀한 제어를 위해 나만의 레이어도 만들고 싶다면 어떻게 할까요. 나만의 모델을 만들려면 케라스 Model 클래스를 상속받은 클래스를 만들었지만, 나만의 레이어를 만들려면 케라스 Layer 클래스를 상속받은 클래스를 만들면 됩니다. Layer를 상속받는 방법은 Model의 방법과 매우 유사합니다. 템플릿은 다음과 같습니다.

```python
import tensorflow as tf
from tensorflow.keras import layers

class MyLayer(layers.Layer):
    
    def __init__(self, num_outputs):
        super(MyLayer, self).__init__()
        
        self.num_outputs = num_outputs
        
        # ...
        
    def build(self, input_shape)
        # create variables
        
    @tf.function
    def call(self, inputs, training=False):
        # operation using variables
```

커스텀 Layer 클래스의 생성자에서도 마찬가지로 상위 클래스인 케라스 Layer의 생성자를 호출해 주어야 합니다.

Layer의 핵심인 weight들의 생성은 생성자에서 해도 되며 build() 메소드를 따로 만들어서 거기서 해줘도 됩니다. 모델 클래스의 경우에도 ```build``` 메소드가 있지만, 생성자에서 모델 구조를 만들어주죠. 하지만, Layer클래스에서는 생성자에서 모두 하기에는 조금 무리가 있습니다. 바로 input_shape을 알아야 weight들을 정의하던 말던 하니까 말입니다. 따라서 보통 layer에선 생성자에서보단 ```build```에서 weight들을 정의합니다. ```build```는 기본적으로 input_shape을 인자로 받아야 합니다.

weight를 만드는 방법은 두 가지가 있습니다.
1. ```tf.Variable()``` 이용
2. ```self.add_weight()``` 이용

tf.Variable은 우리가 직접 텐서플로 variable을 만들어 주는 방식으로, 매우 low-level이 되겠네요. 반면, layer의 멤버 메소드로 있는 ```add_weight```메소드는 tensorflow 1.x 버전의 ```get_variable```과 유사하다고 생각하시면 되겠습니다. 직접 텐서플로 variable을 생성하는 것보다 내부적으로 생성하게 두는 것이죠. 두 방식의 차이점은 텐서플로의 name_scope를 관리하는지에 대한 차이인 듯 한데... 자세히는 모르겠습니다. 어쩄든 첫번째 방식은 개발자가 직접 텐서플로 variable을 생성하는 것이고, 두번쨰 방식은 텐서플로 variable의 생성을 Layer 클래스에게 맡기는 것입니다. 잘 모를때는 두번째 방식을 이용합시다(단점은 variable의 이름을 주는게 필수..).

참고로 ```@tf.function```은 call메소드에만 붙여주거나 아예 붙이지 말아야 합니다. call에 ```@tf.function```을 붙여주면 forward propagation이 조금 더 빠릅니다. 다만, 귀찮은 점이 조금 생길 수 있습니다. 불편하시면 모든 코드에서 아예 ```@tf.function```을 빼는 방법도 있습니다. ```@tf.function```은 해당 함수를 그래프로 빌드시키는 역할을 하는데, 그래프로 빌드한 후 인풋을 그래프에 통과시키므로 속도가 빨라집니다(당연히 바이너리 코드상태로 그래프를 빌드할것이고, 파이썬보단 빠르겠죠).

이제 진짜 레이어를 만들어 봅시다. 다음은 leaky relu를 포함하는 dense 레이어를 직접 만들어 본 것입니다.

In [99]:
import tensorflow as tf
from tensorflow.keras import layers

class DenseWithLeakyReLU(layers.Layer):
    
    def __init__(self, num_out_features, name="", *args, **kwargs):
        super(DenseWithLeakyReLU, self).__init__(name=name, *args, **kwargs)
        
        self.num_out_features = num_out_features
        
    def build(self, input_shape):
        self.W = self.add_weight("W", shape=(input_shape[-1], self.num_out_features))
        self.b = self.add_weight("b", shape=(1, self.num_out_features))
        
    @tf.function
    def call(self, inputs, training=False):
        z = tf.matmul(inputs, self.W) + self.b
        x = tf.nn.leaky_relu(z)
        return x

먼저, 생성자로, 다른 케라스 내장 레이어들과 마찬가지로 output feature의 개수를 인자로 받습니다. 그리고 뒤에 이상한 것들이 더 붙었는데요, 먼저, name은 이 레이어에 이름을 주기 위한 것으로, 상위 클래스(Layer)의 생성자로 포워딩(그대로 전달)해 줄 것입니다. 여기까진 그렇다 치고, ```*args, **kwargs```는 뭐냐 하고 물으실 수 있는데, 이건 파이썬 문법으로, \*args는 값을 나열한 것을 list로 묶는 역할, \*\*kwargs는 key=value 로 나열한 것을 dictionary로 묶는 역할을 합니다. 예를들어, 다음 함수가 있다고 칩시다.

```python
def func(name, *args, **kwargs):
    pass
```

그리고 다음처럼 호출해보면,
```python
func("NAME", 1, 2, 3, a="a", b="b")
```

```name```에는 "NAME"이 들어가고 ```args```에는 \[1, 2, 3\], ```**kwargs```에는 {"a": "a", "b": "b"}가 들어갑니다. 그래서 ```func```내부에서 ```args[0], kwargs["b"]```이렇게 접근이 가능하죠.

이런 것은 케라스 Model의 생성자에도 써먹어도 됩니다. 좀이따 나올 코드처럼 말이죠.

그 다음으로는 ```build```메소드를 봅시다. 이 메소드는 기본적으로 input_shape을 받아야만 합니다. 보통 미리 shape을 알 수 없는 variable들을 여기서 선언해 주고 shape을 미리 알 수 있다면 생성자에 선언해 줘도 됩니다.

```call```메소드는 Model의 ```call```과 역할이 비슷합니다. Model에서의 ```call```은 만들어둔 레이어들에 inputs을 넣어 짜맞추는 역할이라면, Layer에서의 ```call```은 inputs을 만들어 놓은 weight들과 연산해 주는 것입니다.

(build메소드는 @tf.function을 붙이면 안됩니다.)

다음으로, 만든 레이어를 이용한 모델을 만들어 보고 제대로 되는지 보고 마치도록 하겠습니다.

In [100]:
import tensorflow as tf
from tensorflow.keras import models, layers

class MyNewModel(models.Model):
    
    def __init__(self, name="", *args, **kwargs):
        super(MyNewModel, self).__init__(name=name, *args, **kwargs)
        
        self.classifier = models.Sequential([
            DenseWithLeakyReLU(128, input_shape=(784,)),
            DenseWithLeakyReLU(128),
            layers.Dense(10, activation=tf.nn.softmax)
        ])
        
    @tf.function
    def call(self, inputs, training=False):
        x = self.classifier(inputs, training=training)
        return x

In [101]:
mymodel = MyNewModel()
mymodel.build(input_shape=(None, 784))
mymodel.summary()

Model: "my_new_model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_12 (Sequential)   (None, 10)                118282    
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [102]:
EPOCHS = 10
ETA = 1e-3
BATCH_SIZE = 64

In [103]:
optimizer = optimizers.Adam(learning_rate=ETA)
criterion = losses.SparseCategoricalCrossentropy()

train_acc = metrics.SparseCategoricalAccuracy()
test_acc = metrics.SparseCategoricalAccuracy()

train_loss = metrics.Mean()
test_loss = metrics.Mean()

In [104]:
for e in range(EPOCHS):
    x_shuffled, y_shuffled = shuffle(x_train, y_train)
    
    for x_batch, y_batch in next_batch(x_shuffled, y_shuffled, BATCH_SIZE):
        loss, preds, grads = inference(mymodel, criterion, x_batch, y_batch, True)
        
        # 이것이, 결과를 축적해서 나중에 합쳐주는 metric입니다.
        train_loss(loss)
        train_acc(y_batch, preds)
        
        backward(optimizer, grads, mymodel.trainable_variables)
        
    for x_batch, y_batch in next_batch(x_test, y_test, BATCH_SIZE):
        loss, preds = inference(mymodel, criterion, x_batch, y_batch, False)
        
        test_loss(loss)
        test_acc(y_batch, preds)
        
    print(f"Epochs: {e+1}/{EPOCHS}")
    print(f"Train loss: {train_loss.result():.8f}")
    print(f"Train acc: {train_acc.result():.4f}")
    print(f"Test loss: {test_loss.result():.8f}")
    print(f"Test acc: {test_acc.result():.4f}")
    print()
    
    # 축적한 놈들을 없애버립니다.
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()

Epochs: 1/10
Train loss: 0.33923173
Train acc: 0.8962
Test loss: 0.16758858
Test acc: 0.9508

Epochs: 2/10
Train loss: 0.16397022
Train acc: 0.9491
Test loss: 0.13615265
Test acc: 0.9566

Epochs: 3/10
Train loss: 0.12060469
Train acc: 0.9636
Test loss: 0.11458984
Test acc: 0.9655

Epochs: 4/10
Train loss: 0.09962751
Train acc: 0.9689
Test loss: 0.09078751
Test acc: 0.9724

Epochs: 5/10
Train loss: 0.08469982
Train acc: 0.9735
Test loss: 0.10330206
Test acc: 0.9668

Epochs: 6/10
Train loss: 0.07369034
Train acc: 0.9765
Test loss: 0.09014604
Test acc: 0.9725

Epochs: 7/10
Train loss: 0.06642648
Train acc: 0.9782
Test loss: 0.10683333
Test acc: 0.9676

Epochs: 8/10
Train loss: 0.05872803
Train acc: 0.9813
Test loss: 0.08457179
Test acc: 0.9739

Epochs: 9/10
Train loss: 0.05262538
Train acc: 0.9832
Test loss: 0.12411152
Test acc: 0.9628

Epochs: 10/10
Train loss: 0.04914222
Train acc: 0.9836
Test loss: 0.09474929
Test acc: 0.9718



**워닝은 알아보고 업뎃하겠습니다.**